### Archive LTO Course Enrollments ###
The purpose of this file is to archive the enrollments of users in courses in Docebo to save their progress, but remove the course from their view in their dashboards. This is generally used for LTO courses that are temporary, but still need to be completed by kiosk employees.

This script has two main functions. You should only run one or the other, but not both in the same execution.
- The "Single Course Removal" targets a single course at a time and only includes users that are enrolled in that course to perform an API call against.
- The "Multi-course Removal" targets multiple courses and attempts to remove all users that are enrolled in any one of those courses. This function is better to use if you have many users that overlap in multiple courses. Don't use this one if the users are not in the majority of the courses you are trying to archive.

In [ ]:
import pandas as pd
import requests
import json
import math
import datetime as dt
import os
from dotenv import load_dotenv

In [ ]:
curDate = dt.datetime.today()
dateString = curDate.strftime('%Y-%m-%d')

In [ ]:
#Client information
load_dotenv()
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [ ]:
#Get authorization token
#Data payload for token request
token_data = {'client_id':client_id,'client_secret':client_secret,'grant_type':'password', 'username':username, 'password':password, 'scope':'api'}

p1 = requests.post('https://scooterscoffee.docebosaas.com/oauth2/token', data=token_data)

p1_result = json.loads(p1.text)

print(p1_result)

In [ ]:
#Access token information
access_token = p1_result['access_token']

In [ ]:
#Headers 
headers = {'Accept':'application/json','Client-ID':client_id, 'Authorization':'Bearer ' + access_token}

In [ ]:
#Get list of All Courses

#API endpoint for getting Courses
courses_URL = 'https://scooterscoffee.docebosaas.com/learn/v1/courses'
pageSize = 200

requestPayload1 = requests.get(courses_URL + '?page_size=' + str(pageSize), headers=headers)
requestJSON1 = requestPayload1.json()

#Get total number of users
recordCount = int(requestJSON1['data']['total_count'])

print(requestJSON1['data']['total_count'])

#Calculate the number of loops needed to retrieve all records
loopCounter = recordCount / pageSize

#Round up loopCounter
looper = math.ceil(loopCounter)

In [ ]:
#Query all possible records for All Courses

#Initialize dataframe
courses_df = pd.DataFrame()

#Initialize page number
pageNum = 1

#Get all pages of data
for i in range(looper):
    requestPayload2 = requests.get(courses_URL + '?page_size=' + str(pageSize) + '&page=' + str(pageNum), headers=headers)
    requestJSON2 = requestPayload2.json()
    print("Page " + str(pageNum) + " has finished its call.")
    pageNum += 1
    itemsDict = requestJSON2['data']['items']
    itemsDict_df = pd.DataFrame.from_dict(itemsDict)
    courses_df = pd.concat([courses_df, itemsDict_df], ignore_index=True)
    
print("All records pulled.")

#Expand out items from "category" field to get appropriate courses
#Category ID
courses_df['category_id'] = courses_df.apply(lambda x: x['category']['id'], axis=1)
#Category Name
courses_df['category_name'] = courses_df.apply(lambda x: x['category']['name'], axis=1)

In [ ]:
#Import course enrollments to be archived
#This file is generated by exporting the "Archive LTO Enrollments" report from Scooterversity. The courses you want to target need to be specified on the "Filters" tab of the report before exporting. The filepath will also need to be changed to import the "Archive LTO Enrollments" file into this notebook.
archive_LTO_enrollments_df = pd.read_csv("C:\\Users\\michael.deal\\Downloads\\Archive_LTO_Enrollments.csv")

#Create Course List
remove_course_list = list(archive_LTO_enrollments_df['Course Internal ID'].drop_duplicates())

print(remove_course_list)

In [ ]:
#These lists are built using the "Course Internal ID" from Docebo. The user list is created to identify the course ID the users belong to. The user lists are only needed when using the "Single Course Removal" script. The "remove_users_list" is used when using the "Mutli-course Removal" script
#Completed: 271, 272, 236, 235, 234, 255, 273, 105, 118, 80, 239, 277, 282

#Build user lists
# c_54_df = archive_LTO_enrollments_df[archive_LTO_enrollments_df['Course Internal ID'] == 54]
# c_54_list = list(c_54_df['User unique ID'])

#Build All Courses User List
remove_users_list = list(archive_LTO_enrollments_df['User unique ID'].drop_duplicates())



In [ ]:
# #Single Course Removal script

# #!!! When using this script, remember to update the archive_post_url and cur_list variables to use the targeted course ID. Also be sure the user lists built in the previous step are utilizing the same Course Internal IDs as in the variables just mentioned.

# #Delete statement for API call. Users string formatting with list of Power Users with assigned users. The {0} in the url is the substituted value.
# archive_post_url = 'https://scooterscoffee.docebosaas.com/enrollment/v1/courses/54/users/{0}/archive'

# #Current list
# curList = c_54_list

# #Additional payload information. Must pass "All:true" to remove all users on the delete request or it won't remove any. This must be in a json.dumps() to accomodate Docebo.
# dumpText = {}
# dumpText["reset_tracking"] = False
# dumpText["enrollment_mode"] = "delete"

# payload = json.dumps(dumpText)

# #Create variables for list counting in print
# curRecord = 1
# listLen = len(curList)

# #Loop through list of user IDs and substitute them into the DELETE request each time.
# for user in curList:
#     tempArchiveURL = archive_post_url.format(user)
#     requestPayload1 = requests.post(tempArchiveURL, data=payload, headers=headers)
#     requestJSON1 = requestPayload1.json()
#     print("API request: " + tempArchiveURL)
#     print(requestPayload1)
#     print("Finished record "+ str(curRecord) + " of " + str(listLen))
#     curRecord += 1
    

In [ ]:
#Multi-course Removal script

#Run loop to remove assigned users from all Power Users

#Delete statement for API call. Users string formatting with list of Power Users with assigned users. The {0} is the course Course Internal ID, and the {1} is the User unique ID.
archive_post_url = 'https://scooterscoffee.docebosaas.com/enrollment/v1/courses/{0}/users/{1}/archive'

#Remove User List
ruList = remove_users_list

#Remove Course List
rcList = remove_course_list

#Additional payload information. Must pass "All:true" to remove all users on the delete request or it won't remove any. This must be in a json.dumps() to accomodate Docebo.
dumpText = {}
dumpText["reset_tracking"] = False
dumpText["enrollment_mode"] = "delete"

payload = json.dumps(dumpText)

#Create variables for list counting in print
curUser = 1
userListLen = len(ruList)
curCourse = 1
courseListLen = len(rcList)

#Loop through list of user IDs and substitute them into the DELETE request each time.
for course in rcList:
    for user in ruList:
        tempArchiveURL = archive_post_url.format(course, user)
        requestPayload1 = requests.post(tempArchiveURL, data=payload, headers=headers)
        requestJSON1 = requestPayload1.json()
        print("API request: " + tempArchiveURL)
        print(requestPayload1)
        print("Finished record "+ str(curUser) + " of " + str(userListLen) + "; (Course " + str(curCourse) + " of " + str(courseListLen) + ")")
        curUser += 1
    #Reset counter for User
    curUser = 1
    curCourse += 1
print("Update completed.")